In [17]:
# create image dataset from folder dataset/jajanan/test and dataset/jajanan/train using tf.data.Dataset also use num_parallel_calls to load image faster

import tensorflow as tf
import pathlib
import os

# get path of dataset
data_dir = pathlib.Path('dataset/jajanan')

# get all image path
image_path = list(data_dir.glob('*/*/*'))
image_path = [str(path) for path in image_path]

# get all label name
label_name = sorted(item.name for item in data_dir.glob('train/*') if item.is_dir())

# get all label index
label_index = dict((name, index) for index, name in enumerate(label_name))

# get all label
image_label = [label_index[pathlib.Path(path).parent.name] for path in image_path]

# create dataset
dataset = tf.data.Dataset.from_tensor_slices((image_path, image_label))

# load image
def load_image(image_path, image_label):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])
    image = tf.cast(image, tf.float32)
    image = image / 255.0
    return image, image_label

# load image using parallel
dataset = dataset.map(load_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)

# shuffle dataset
dataset = dataset.shuffle(1000)

# split dataset 80% for train and 20% for test
train_dataset = dataset.take(80)
test_dataset = dataset.skip(80)

# batch dataset
train_dataset = train_dataset.batch(32)
test_dataset = test_dataset.batch(32)

# prefetch dataset
train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)
test_dataset = test_dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [18]:
import tensorflow_hub as hub
ResNet_V2_50 = 'https://tfhub.dev/google/imagenet/resnet_v2_50/classification/5'
Inception_V3 = 'https://tfhub.dev/google/imagenet/inception_v3/classification/5'
MobileNet_V3_Large = 'https://tfhub.dev/google/imagenet/mobilenet_v3_large_100_224/classification/5'
Inception_ResNet_V2 = 'https://tfhub.dev/google/imagenet/inception_resnet_v2/classification/5'
NASNet_Large = 'https://tfhub.dev/google/imagenet/nasnet_large/classification/5'


In [15]:
# use GPU
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
  except RuntimeError as e:
    print(e)

# use XLA to optimize performance
tf.config.optimizer.set_jit(True)

# create model
# model_ResNet = tf.keras.Sequential([
#     hub.KerasLayer(ResNet_V2_50, trainable = False, input_shape = (224,224,3), name = 'Resnet_V2_50'),
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(1024, activation = 'relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
#     tf.keras.layers.Dropout(0.4),
#     tf.keras.layers.Dense(17, activation = 'softmax', name = 'Output_layer')
# ])

# model_Inception = tf.keras.Sequential([
#     hub.KerasLayer(Inception_V3, trainable = False, input_shape = (224,224,3), name = 'Inception_V3'),
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(1024, activation = 'relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
#     tf.keras.layers.Dropout(0.4),
#     tf.keras.layers.Dense(17, activation = 'softmax', name = 'Output_layer')
# ])

# model_MobileNet = tf.keras.Sequential([
#     hub.KerasLayer(MobileNet_V3_Large, trainable = False, input_shape = (224,224,3), name = 'MobileNet_V3_Large'),
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(1024, activation = 'relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
#     tf.keras.layers.Dropout(0.4),
#     tf.keras.layers.Dense(17, activation = 'softmax', name = 'Output_layer')
# ])

model_Inception_ResNet = tf.keras.Sequential([
    hub.KerasLayer(Inception_ResNet_V2, trainable = False, input_shape = (224,224,3), name = 'Inception_ResNet_V2'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation = 'relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(17, activation = 'softmax', name = 'Output_layer')
])

# model_NASNet = tf.keras.Sequential([
#     hub.KerasLayer(NASNet_Large, trainable = False, input_shape = (331,331,3), name = 'NASNet_Large'),
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(1024, activation = 'relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
#     tf.keras.layers.Dropout(0.4),
#     tf.keras.layers.Dense(17, activation = 'softmax', name = 'Output_layer')
# ])

Physical devices cannot be modified after being initialized


KeyboardInterrupt: 